In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

import keras
from keras import layers
import tensorflow as tf

import numpy as np

from modules.models.VAE import VAE
from modules.models.VAEFull import VAEFull

import matplotlib
import matplotlib.pyplot as plt
# from barviz import Simplex
# from barviz import Collection

from modules.encoding import ProteinOneHotEncoder

import sys

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)



2024-03-24 00:34:05.779396: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 00:34:05.784186: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 00:34:05.787047: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

### 1.2 - Parameters

In [4]:
scale      = 1
seed       = 123
models_dir = './run'

## Step 3 - Reload best model

In [5]:
vector_size = 200
alphabet_size = 18

vae_full = VAEFull(200, 18)
vae_full.vae.reload(f'{models_dir}/vae-one-hot-{vector_size}.keras')

2024-03-24 00:34:06.109298: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 00:34:06.112591: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 00:34:06.115468: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Reloaded.


In [6]:
vae = vae_full.vae


## Step 4 - Sequence reconstruction

In [7]:
z = tf.random.normal(shape=[16, 2])

x_reconst         = vae_full.decoder.predict(z,      verbose=0)

latent_dim        = z.shape[1]

# labels=[ str(np.round(z[i],1)) for i in range(10) ]
# fidle.utils.subtitle('Originals :')
# fidle.scrawler.images(x_show,    None, indices='all', columns=10, x_size=2,y_size=2, save_as='01-original')
# fidle.utils.subtitle('Reconstructed :')
# fidle.scrawler.images(x_reconst, None, indices='all', columns=10, x_size=2,y_size=2, save_as='02-reconstruct')


2024-03-24 00:34:07.584459: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


In [11]:
encoder = ProteinOneHotEncoder()

In [26]:
x_reconst.shape

(16, 200, 18, 1)

In [25]:
x_reconst_sm = keras.activations.softmax(x_reconst)

AttributeError: 'tuple' object has no attribute 'rank'

In [23]:
x_sample = x_reconst[0].reshape((200, 18))

In [29]:
x_sample.shape

(18,)

In [27]:
keras.activations.softmax(x_sample)

AttributeError: 'tuple' object has no attribute 'rank'

In [24]:
protein = encoder.decode(x_sample)

(18,)


KeyError: (0.5000638, 0.4997755, 0.50001186, 0.49986854, 0.49984217, 0.49976987, 0.4999256, 0.49970847, 0.4997323, 0.49981827, 0.49988732, 0.49986592, 0.49973992, 0.49973765, 0.49981862, 0.49991158, 0.49998307, 0.49986622)

In [15]:
di = {}
di[tuple(np.zeros((10)))] = 0

## Step 5 - Visualizing the latent space

In [9]:
n_show = min( 20000, len(x_data) )

# ---- Select images

x_show, y_show = fidle.utils.pick_dataset(x_data,y_data, n=n_show)

# ---- Get latent points

z_mean, z_var, z = vae.encoder.predict(x_show, verbose=0)

NameError: name 'x_data' is not defined

### 5.1 - Classic 2d visualisaton

In [ ]:
fig = plt.figure(figsize=(14, 10))
plt.scatter(z[:, 2] , z[:, 4], c=y_show, cmap= 'tab10', alpha=0.5, s=30)
plt.colorbar()
fidle.scrawler.save_fig('03-Latent-space')
plt.show()

### 5.2 - Simplex visualisaton

In [ ]:
if latent_dim<4:

    print('Sorry, This part can only work if the latent space is greater than 3')

else:

    # ---- Softmax rescale
    #
    zs = np.exp(z)/np.sum(np.exp(z),axis=1,keepdims=True)
    # zc  = zs * 1/np.max(zs)

    # ---- Create collection
    #
    c = Collection(zs, colors=y_show, labels=y_show)
    c.attrs.markers_colormap     = {'colorscale':'Rainbow','cmin':0,'cmax':latent_dim}
    c.attrs.markers_size         = 5
    c.attrs.markers_border_width = 0
    c.attrs.markers_opacity      = 0.8

    s = Simplex.build(latent_dim)
    s.attrs.width  = 1000
    s.attrs.height = 1000
    s.plot(c)

## Step 6 - Generate from latent space (latent_dim==2)

In [ ]:
if latent_dim>2:

    print('Sorry, This part can only work if the latent space is of dimension 2')

else:

    grid_size   = 14
    grid_scale  = 1.

    # ---- Draw a ppf grid

    grid=[]
    for y in scipy.stats.norm.ppf(np.linspace(0.99, 0.01, grid_size),scale=grid_scale):
        for x in scipy.stats.norm.ppf(np.linspace(0.01, 0.99, grid_size),scale=grid_scale):
            grid.append( (x,y) )
    grid=np.array(grid)

    # ---- Draw latentspoints and grid

    fig = plt.figure(figsize=(12, 10))
    plt.scatter(z[:, 0] , z[:, 1], c=y_show, cmap= 'tab10', alpha=0.5, s=20)
    plt.scatter(grid[:, 0] , grid[:, 1], c = 'black', s=60, linewidth=2, marker='+', alpha=1)
    fidle.scrawler.save_fig('04-Latent-grid')
    plt.show()

    # ---- Plot grid corresponding images

    x_reconst = vae.decoder.predict([grid])
    fidle.scrawler.images(x_reconst, indices='all', columns=grid_size, x_size=0.5,y_size=0.5, y_padding=0,spines_alpha=0.1, save_as='05-Latent-morphing')



In [ ]:
fidle.end()

---
<img width="80px" src="../fidle/img/logo-paysage.svg"></img>